reference: http://trap.ncirl.ie/1845/1/briangibbons.pdf

# 1. Import Packages

In [1]:
from urllib.request import urlopen
import json
#from bs4 import BeautifulSoup
#from lxml import html
#import requests
import pandas as pd
#import numpy as np
from openpyxl import load_workbook
#import os

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline

# 2. Fuction

In [2]:
def writing_in_excel(path, df, sheet_name):
    writer = pd.ExcelWriter(path, datetime_format='m/d/yyyy')
    df.to_excel(writer, sheet_name = sheet_name, index=False, freeze_panes = (0,1))
    writer.save()
    writer.close()

In [3]:
def writing_sheets_in_excel(path, df, sheet_name):
    book = load_workbook(path)
    writer = pd.ExcelWriter(path, engine = 'openpyxl', date_format='m/d/yyyy')
    writer.book = book
    df.to_excel(writer, sheet_name = sheet_name, index=False, freeze_panes = (0,1))
    writer.save()
    writer.close()

In [4]:
def drop_xy(df):
    # list comprehension of the cols that end with '_y'
    to_drop = [x for x in df if x.endswith('_y')]
    df.drop(to_drop, axis=1, inplace=True)
    
    #to_replace = [x for x in df if x.endswith('_x')]
    df.columns = [c.lower().replace('_x', '') for c in df.columns] 

In [5]:
def creating_excel_by_keys(path, html):
    path = path
    raw_html = urlopen(html)
    data = json.load(raw_html)
    keys = list(data.keys())
    i = 0
    while i < len(keys):
        try:
            name = keys[i]
            df1 = pd.DataFrame(data[keys[i]])
            writing_in_excel(path + name + '.xlsx', df1, name)
        except:
            pass
        i += 1


# Test if crawler works

In [6]:
# info about bootstrap (overall)
bootstrap_html = urlopen("https://fantasy.premierleague.com/api/bootstrap-static/")
bootstrap = json.load(bootstrap_html)

In [12]:
# info about one player

player_html = urlopen("https://fantasy.premierleague.com/api/element-summary/1/")
player_raw = json.load(player_html)

In [13]:
bootstrap.keys()
player_raw.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

dict_keys(['fixtures', 'history', 'history_past'])

In [9]:
len(bootstrap['elements'])

592

In [10]:
# From Bootstrap

events = pd.DataFrame(bootstrap['events'])
#game_settings = pd.DataFrame(bootstrap['game_settings'])
phases =  pd.DataFrame(bootstrap['phases']) 
teams =  pd.DataFrame(bootstrap['teams']) 
#total_players = pd.DataFrame(bootstrap['total_players'])
elements =  pd.DataFrame(bootstrap['elements']) 
element_stats =  pd.DataFrame(bootstrap['element_stats']) 
element_types =  pd.DataFrame(bootstrap['element_types'])

fixtures = pd.DataFrame(player_raw['fixtures'])
history = pd.DataFrame(player_raw['history'])
history_past = pd.DataFrame(player_raw['history_past'])

In [50]:
#events
#phases
#teams
#elements

#element_stats
#element_types



0      a
1      d
2      a
3      a
4      a
      ..
584    a
585    a
586    a
587    a
588    a
Name: status, Length: 589, dtype: object

In [15]:
##### From Player_raw ######

fixtures = pd.DataFrame(player_raw['fixtures'])
history = pd.DataFrame(player_raw['history'])
history_past =  pd.DataFrame(player_raw['history_past']) 
    
fixtures['element'] = 1

fixtures
#history.iloc
#history_past

,code,team_h,team_h_score,team_a,team_a_score,event,finished,minutes,provisional_start_time,kickoff_time,event_name,is_home,difficulty,element
0,1059915,7,None,1,None,22,False,0,False,2020-01-11T12:30:00Z,Gameweek 22,False,2,1
1,1059922,1,None,15,None,23,False,0,False,2020-01-18T15:00:00Z,Gameweek 23,True,3,1
2,1059939,6,None,1,None,24,False,0,False,2020-01-21T20:15:00Z,Gameweek 24,False,4,1
3,1059943,5,None,1,None,25,False,0,False,2020-02-02T14:00:00Z,Gameweek 25,False,3,1
4,1059952,1,None,13,None,26,False,0,False,2020-02-16T16:30:00Z,Gameweek 26,True,2,1
5,1059962,1,None,8,None,27,False,0,False,2020-02-23T16:30:00Z,Gameweek 27,True,2,1
6,1059976,11,None,1,None,28,False,0,False,2020-03-01T14:00:00Z,Gameweek 28,False,5,1
7,1059982,1,None,19,None,29,False,0,False,2020-03-07T15:00:00Z,Gameweek 29,True,2,1
8,1059994,4,None,1,None,30,False,0,False,2020-03-14T15:00:00Z,Gameweek 30,False,3,1
9,1060009,16,None,1,None,31,False,0,False,2020-03-21T15:00:00Z,Gameweek 31,False,2,1


In [11]:
##### From Player_raw ######

fixtures = pd.DataFrame(player_raw['fixtures'])
history = pd.DataFrame(player_raw['history'])
history_past =  pd.DataFrame(player_raw['history_past']) 
    
fixtures
#history.iloc
#history_past

,code,team_h,team_h_score,team_a,team_a_score,event,finished,minutes,provisional_start_time,kickoff_time,event_name,is_home,difficulty
0,1059922,1,None,15,None,23,False,0,False,2020-01-18T15:00:00Z,Gameweek 23,False,4
1,1059937,15,None,11,None,24,False,0,False,2020-01-21T19:30:00Z,Gameweek 24,True,4
2,1059944,7,None,15,None,25,False,0,False,2020-02-01T15:00:00Z,Gameweek 25,False,2
3,1059959,15,None,3,None,26,False,0,False,2020-02-09T14:00:00Z,Gameweek 26,True,2
4,1059969,15,None,4,None,27,False,0,False,2020-02-22T15:00:00Z,Gameweek 27,True,2
5,1059973,2,None,15,None,28,False,0,False,2020-02-29T15:00:00Z,Gameweek 28,False,2
6,1059989,15,None,14,None,29,False,0,False,2020-03-07T15:00:00Z,Gameweek 29,True,2
7,1059997,13,None,15,None,30,False,0,False,2020-03-14T15:00:00Z,Gameweek 30,False,3
8,1060006,12,None,15,None,31,False,0,False,2020-03-21T15:00:00Z,Gameweek 31,False,4
9,1060019,15,None,17,None,32,False,0,False,2020-04-04T14:00:00Z,Gameweek 32,True,4


# Defining a scraper

In [16]:
def bootstrap_scraper():
    
    # info about bootstrap (overall)
    bootstrap_html = urlopen("https://fantasy.premierleague.com/api/bootstrap-static/")
    bootstrap = json.load(bootstrap_html)
    
    events = pd.DataFrame(bootstrap['events'])
    #game_settings = pd.DataFrame(bootstrap['game_settings'])
    phases =  pd.DataFrame(bootstrap['phases']) 
    teams =  pd.DataFrame(bootstrap['teams']) 
    #total_players = pd.DataFrame(bootstrap['total_players'])
    elements =  pd.DataFrame(bootstrap['elements']) 
    element_stats =  pd.DataFrame(bootstrap['element_stats']) 
    element_types =  pd.DataFrame(bootstrap['element_types'])
    
    
    select = 'D'

    events.to_csv(select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\2019//events2019.csv')
    #game_settings.to_csv(select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\game_settings2019.xlsx')
    phases.to_csv(select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\phases2019.csv')
    teams.to_csv(select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\teams2019.csv')
    elements.to_csv(select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\elements2019.csv')
    element_stats.to_csv(select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\element_stats2019.csv')
    element_types.to_csv(select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\element_types2019.csv')


In [19]:
def player_data_scraper():
    
    i=1
    history_list = []
    fixtures_list = []
    
    while i < 600:
        try:
            player_html = urlopen("https://fantasy.premierleague.com/api/element-summary/" + str(i) + '/')            
            player_raw = json.load(player_html)
                   
            history = pd.DataFrame(player_raw['history'])
            fixtures = pd.DataFrame(player_raw['fixtures'])

            
            fixtures['element'] = i
            
            history_list.append(history)
            fixtures_list.append(fixtures)
            
            
            history_df = pd.concat(history_list)
            history_df = pd.DataFrame(history_df)
            
            fixtures_df = pd.concat(fixtures_list)
            fixtures_df = pd.DataFrame(fixtures_df)
            #print(i)
            
        except:
            pass
            
    
        print (i)
        i += 1
    
    return history_df, fixtures_df
    

# Run the scrapers

In [23]:
bootstrap_scraper()
history_df, fixtures_df = player_data_scraper()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [24]:
select = 'D'
history_df.to_csv(select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\history_df.csv')
fixtures_df.to_csv(select + ':\\OneDrive - Georgia State University\\Data Science\\Personal_Project\\FPL_Analysis\\data\\fixture_df.csv')


# Debugging
